In [9]:
import numpy as np
import copy
from itertools import combinations
import pandas as pd
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation

In [10]:
import numpy as np
import pandas as pd
from numpy import random
import scipy.sparse as sp
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import copy
import math
from functools import cmp_to_key

In [11]:
def detect_communities(user_item_matrix):
    # Compute user-user similarity
    user_similarities = np.zeros((user_item_matrix.shape[0], user_item_matrix.shape[0]))
    for i in range(user_item_matrix.shape[0]):
        for j in range(i+1, user_item_matrix.shape[0]):
            user_similarities[i][j] = np.dot(user_item_matrix[i], user_item_matrix[j]) / (np.linalg.norm(user_item_matrix[i]) * np.linalg.norm(user_item_matrix[j]))
            user_similarities[j][i] = user_similarities[i][j]

    # Run overlapping community detection algorithm
    overlapping_communities = []
    for i in range(user_item_matrix.shape[0]):
        community_found = False
        for community in overlapping_communities:
            if i in community:
                community_found = True
                break
        if not community_found:
            new_community = {i}
            for j in range(i+1, user_item_matrix.shape[0]):
                if user_similarities[i][j] > 0.2:
                    new_community.add(j)
            overlapping_communities.append(new_community)

    return overlapping_communities, user_similarities


def PMF(user_item_matrix, num_iterations=100, learning_rate=0.01, num_latent_factors=2, alpha=0.01, beta=0.01):
    # Initialize user and item latent factor matrices
    user_factors = np.random.normal(size=(len(user_item_matrix), num_latent_factors))
    item_factors = np.random.normal(size=(len(user_item_matrix[0]), num_latent_factors))

    # Update user and item latent factor matrices using stochastic gradient descent
    for iteration in range(num_iterations):
        for i in range(len(user_item_matrix)):
            for j in range(len(user_item_matrix[0])):
                if user_item_matrix[i][j] != 0:
                    error = user_item_matrix[i][j] - np.dot(user_factors[i], item_factors[j])
                    user_factors[i] += learning_rate * (error * item_factors[j] - alpha * user_factors[i])
                    item_factors[j] += learning_rate * (error * user_factors[i] - beta * item_factors[j])

    return user_factors, item_factors


def sortbyCond(a, b):
    if (a[0] != b[0]):
        return (b[1] - a[1])
    else:
        return b[0] - a[0]


# This function is for training and for calculating rmse score
def update_user_item_rating_matrix(user_id, R_11, community_pmf, communities):
  user_communities = []
  for i, community in enumerate(communities):
      if user_id in community:
        user_communities.append(i)
  for i in user_communities:
    user_ind = 0
    for ind, user in enumerate(communities[i]):
      if user == user_id:
        user_ind = ind
        break
    il = community_pmf[i][user_ind]
    for i_, rating in enumerate(il):
      R_11[user_ind][i_] = max(R_11[user_ind][i_], rating)




# This function returns ton-N items for users where are not seen that item previously
def recommend_items(user_id, user_item_matrix, R_11, num_items=2):
  item_list = [0 for _ in range(len(user_item_matrix[0]))] 
  for i, rating in enumerate(R_11[user_id]):
    if user_item_matrix[user_id][i] == 0:
      if (user_item_matrix[user_id][i]-item_list[i]) > (user_item_matrix[user_id][i]-rating):
        item_list[i] = rating

  recommende_item = []
  for i, rating in enumerate(item_list):
    if rating != 0:
      recommende_item.append((i,rating))

  recommende_item.sort(key = cmp_to_key(sortbyCond))
  if len(recommende_item) < 2:
    return recommende_item
  return recommende_item[0], recommende_item[1]

In [12]:
# we can use this algorithm also to find overlapping communities after finding similarity matrix
# def cpm_algorithm(R, k):
    # Find all k-cliques in the similarity matrix
    # sim_matrix = np.zeros((R.shape[0], R.shape[0]))
    # for i in range(user_item_matrix.shape[0]):
    #     for j in range(i+1, user_item_matrix.shape[0]):
    #         sim_matrix[i][j] = np.dot(R[i], R[j]) / (np.linalg.norm(R[i]) * np.linalg.norm(R[j]))
    #         sim_matrix[j][i] = sim_matrix[i][j]
    # cliques = []
    # n = sim_matrix.shape[0]
    # for i in range(n):
    #     # Find all k-cliques containing node i
    #     nodes = np.where(sim_matrix[i] >= 0.5)[0]
    #     if len(nodes) >= k:
    #         for clique in combinations(nodes, k):
    #             if all(sim_matrix[i, j] >= 0.5 for j in clique):
    #                 cliques.append(set(clique + (i,)))
    
    # # Merge cliques that share at least k-1 nodes
    # communities = []
    # while cliques:
    #     community = set(cliques.pop())
    #     i = 0
    #     while i < len(cliques):
    #         clique = cliques[i]
    #         if len(community & clique) >= k-1:
    #             community |= clique
    #             cliques.pop(i)
    #         else:
    #             i += 1
    #     communities.append(community)
    
    # return communities

In [13]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
# for training 
ratings = pd.read_csv('/content/u.data', sep='\t', names=r_cols,encoding='latin-1')
ratings.drop( "unix_timestamp", inplace = True, axis = 1 )
print(ratings.head(5))
r_cols_ = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
# for testing
ratings_ = pd.read_csv('/content/u1.base', sep='\t', names=r_cols_,encoding='latin-1')
ratings_.drop( "unix_timestamp", inplace = True, axis = 1 )
print(ratings_.shape)
print(ratings.shape)
print(ratings_)

   user_id  movie_id  rating
0      196       242       3
1      186       302       3
2       22       377       1
3      244        51       2
4      166       346       1
(80000, 3)
(100000, 3)
       user_id  movie_id  rating
0            1         1       5
1            1         2       3
2            1         3       4
3            1         4       3
4            1         5       3
...        ...       ...     ...
79995      943      1067       2
79996      943      1074       4
79997      943      1188       3
79998      943      1228       3
79999      943      1330       3

[80000 rows x 3 columns]


In [14]:
ratings_matrix = ratings.pivot_table(index=['user_id'],columns=['movie_id'],values='rating').reset_index(drop=True)
ratings_matrix.fillna( 0, inplace = True )
R = ratings_matrix.values
RR = copy.deepcopy(R)
n_users, n_items = R.shape
ratings_matrix_ = ratings_.pivot_table(index=['user_id'],columns=['movie_id'],values='rating').reset_index(drop=True)
ratings_matrix_.fillna( 0, inplace = True )
R_1 = ratings_matrix_.values
# print(R.shape, R_1.shape)
N,M = R_1.shape

user_item_matrix = copy.deepcopy(R)
for i in range(N):
  for j in range(M):
    user_item_matrix[i][j] = R_1[i][j]
R_11 = copy.deepcopy(user_item_matrix)
# print(user_item_matrix)
# print(R.shape, R_1.shape,user_item_matrix.shape)

# find user similarity and Detect overlapping communities 
communities,m = detect_communities(R)

# Print the detected communities
# for i, community in enumerate(communities):
#     print(f"Community {i+1}: {community}")

community_pmf = []
k = 0
for community in communities:
  R_2 = [] 
  for i, user in enumerate(community):
    R_2.append(user_item_matrix[user])
  #applying pmf for this community
  R_2 = np.array(R_2).tolist()
  user_factor, item_factor = PMF(R_2)
  new_R = np.dot(user_factor, item_factor.T)
  community_pmf.append(new_R)

# update the user item rating matrix with community PMF predicted ratings
for user_id in range(R.shape[0]):
  update_user_item_rating_matrix(user_id, R_11, community_pmf, communities)

# recommend items to the users
for user_id in range(R.shape[0]):
  print(f"Recommended items for user {user_id+1}: {recommend_items(user_id, user_item_matrix, R_11)}")
  if user_id+1 == 500:
    break


Recommended items for user 1: ((385, 11.400702448787685), (984, 11.00091098847643))
Recommended items for user 2: ((612, 14.04533295590802), (1323, 11.164013578284246))
Recommended items for user 3: ((1152, 12.635742812727694), (763, 12.237307540514553))
Recommended items for user 4: ((984, 12.596276420768863), (934, 12.563787980642656))
Recommended items for user 5: ((1185, 10.01906348494968), (1568, 9.864838847526972))
Recommended items for user 6: ((1568, 10.25139066001173), (1185, 10.087465442690222))
Recommended items for user 7: ((1088, 12.281254177635322), (1158, 12.042173085125365))
Recommended items for user 8: ((385, 10.781221579639272), (829, 9.9855695719137))
Recommended items for user 9: ((1185, 11.151126380821516), (1650, 10.221258952832946))
Recommended items for user 10: ((829, 11.222299508393268), (1568, 11.087185464358187))
Recommended items for user 11: ((1568, 10.862296687190886), (1550, 10.66857285396923))
Recommended items for user 12: ((1185, 10.478333340195956),

In [15]:
def root_mean_square(user_item_matrix, R_11):
  n, m = user_item_matrix.shape
  cnt = 0.0
  diff = 0.0
  for i in range(n):
    for j in range(m):
      if user_item_matrix[i][j]>0:
        cnt += 1.0
        diff += (R_11[i][j] - user_item_matrix[i][j]) ** 2
  diff = abs(diff)/cnt
  return diff

In [16]:
from sklearn.metrics import mean_squared_error
from math import sqrt
print(f"CBPMF RMSE Score: {sqrt(root_mean_square(user_item_matrix, R_11))}")
u_1, v_1 = PMF(user_item_matrix)
R_PMF = np.dot(u_1, v_1.T)
print(f"PMF RMSE Score: {sqrt(root_mean_square(user_item_matrix, R_PMF))}")

CBPMF RMSE Score: 0.8139220463239705
PMF RMSE Score: 0.8849669082308035
